In [1]:
import numpy as np
import Loader
import Transformer

C:\Users\Lemon\AppData\Local\Programs\Python\Python38\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
compoundsTrain, smilesTrain, labelsTrain, compoundDataTrain, activitiesTrain = Loader.getTrain(defaultValue=0)
compoundsTest, smilesTest, labelsTest, compoundDataTest, activitiesTest = Loader.getTest(defaultValue=0)
compoundsValidate, smilesValidate, labelsValidate, compoundDataValidate, activitiesValidate = Loader.getValidate(defaultValue=0)

In [4]:
#after transformations are done assign data
dataLabels = labelsTrain[0:20] #only docking and fusion
trainData = compoundDataTrain[:,0:20]
testData = compoundDataTest[:,0:20]
valData = compoundDataValidate[:,0:20]

trainData, testData, valData = Transformer.normalizeData(trainData, testData, valData, newMean=0, newStd=1)
outputData = Transformer.toClassification(activitiesTrain)

In [6]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.losses import MeanSquaredError

In [8]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=3e-2,
    decay_steps=10000,
    decay_rate=1)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

l1Reg = 1e-10

model = Sequential()
model.add(Dense(np.shape(trainData)[1], input_dim=np.shape(trainData)[1], activation='relu', 
                kernel_regularizer = keras.regularizers.L2(l1Reg)))
model.add(Dense(50, activation='relu', kernel_regularizer = keras.regularizers.L1()))
model.add(Dense(80, activation='relu', kernel_regularizer = keras.regularizers.L1(l1Reg)))
model.add(Dense(100, activation='relu', kernel_regularizer = keras.regularizers.L1(l1Reg)))
model.add(Dense(80, activation='relu', kernel_regularizer = keras.regularizers.L1(l1Reg)))
model.add(Dense(60, activation='relu', kernel_regularizer = keras.regularizers.L1(l1Reg)))
model.add(Dense(40, activation='relu', kernel_regularizer = keras.regularizers.L1(l1Reg)))
model.add(Dense(20, activation='relu', kernel_regularizer = keras.regularizers.L1(l1Reg)))
model.add(Dense(10, activation='relu', kernel_regularizer = keras.regularizers.L1(l1Reg)))
model.add(Dense(1, activation='tanh', kernel_regularizer = keras.regularizers.L1(l1Reg)))

model.compile(loss='MeanSquaredError', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(trainData, Transformer.toClassification(activitiesTrain), 
                    validation_data = (valData, Transformer.toClassification(activitiesValidate)), epochs=1600, batch_size=8)

Epoch 1/1600
340/340 [==============================] - 3s 3ms/step - loss: 1.8849 - accuracy: 0.0386 - val_loss: 1.4484 - val_accuracy: 0.2471
Epoch 2/1600
340/340 [==============================] - 1s 2ms/step - loss: 1.2160 - accuracy: 0.1108 - val_loss: 0.9965 - val_accuracy: 0.1029
Epoch 3/1600
340/340 [==============================] - 1s 2ms/step - loss: 0.9350 - accuracy: 0.0913 - val_loss: 0.8854 - val_accuracy: 0.0000e+00
Epoch 4/1600
340/340 [==============================] - 1s 2ms/step - loss: 0.8954 - accuracy: 0.0939 - val_loss: 0.8809 - val_accuracy: 0.0000e+00
Epoch 5/1600
340/340 [==============================] - 1s 2ms/step - loss: 0.8951 - accuracy: 0.0766 - val_loss: 0.8690 - val_accuracy: 0.0000e+00
Epoch 6/1600
340/340 [==============================] - 1s 2ms/step - loss: 0.8856 - accuracy: 0.0880 - val_loss: 0.8633 - val_accuracy: 0.0000e+00
Epoch 7/1600
340/340 [==============================] - 1s 2ms/step - loss: 0.8837 - accuracy: 0.0916 - val_loss: 0.8863

KeyboardInterrupt: 